# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Porciuncula Yamada

Nome: Pedro Henrique de Sousa da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

* Tweets relevantes:
    - Críticas à marca ou produtos
    - Anúncios de eventos
    - Venda de produtos


___
Carregando algumas bibliotecas:

In [631]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
import functools
import operator
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Paula
[nltk_data]     Yamada\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [632]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Paula Yamada\OneDrive\Área de Trabalho\2021.2\cdados\cdados-projeto-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [633]:
filename = 'Apple.xlsx'

In [634]:
train = dft = pd.read_excel('Apple.xlsx', sheet_name='Treinamento')
test = dft = pd.read_excel('Apple.xlsx', sheet_name='Teste')

___
## Classificador automático de sentimento


Apple é uma empresa norte-americana que recentemente anunciou o lançamento de diversos produtos como o iPhone 13 e o Apple Watch 7.

A classificação foi feita considerando se o tweet tem relação com a marca ou se faz parte de uma discussão relacionada à empresa: opinião, crítica ou afim.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
Aqui criamos funções com os seguintes intuitos:
- Remoção de pontuação (`função I`);
- Correção de espaços entre emojis (`função II`);
- Remoção de palavras de parada (ou *stopwords*) (`função III`);

Ao final da montagem, também aplicamos a Suavização de Laplace.

In [635]:
import re 

# FUNÇÃO I
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    pont = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(pont)

    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '', text_subbed)

    return text_subbed

# FUNÇÃO II
def separa_emoji(text):
    """
        Função que separa emojis em frases
    """
    em_split_emoji = emoji.get_emoji_regexp().split(text)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    return em_split

# FUNÇÃO III
def remove_stopwords(lista):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    l = list()
    for p in lista:
        if not p in stopwords:
            l.append(p)
    return l

___
### Obtenção de valores a partir dos DataFrames

In [636]:
# Criando uma série para contabilizar as frequências relativas de
# todos os tweets da planilha 'Treinamento'

lista = list()
for i in range(train.shape[0]):
    lista += remove_stopwords(separa_emoji(cleanup(train.Treinamento[i])))
serie_twt = pd.Series(lista)
twt_relat = serie_twt.value_counts(True)

In [637]:
# Elaborando tabelas separadas para os tweets
# classificados como 'relevantes' ou 'irrelevantes'
bool_ = train.isin([1])

filtro_r = bool_['Classificação'] == True
filtro_i = bool_['Classificação'] == False

relevantes = train.loc[filtro_r, :]
irrelevantes = train.loc[filtro_i, :]

In [638]:
# Criando uma série para contabilizar as frequências relativas de
# tweets relevantes da planilha 'Treinamento'
li_rel = list()

for i in range(relevantes.shape[0]):
    li_rel += remove_stopwords(separa_emoji(cleanup(relevantes.Treinamento[relevantes.index[i]])))

serie_rel = pd.Series(li_rel)
rel_relat = serie_rel.value_counts(True)

In [639]:
# Criando uma série para contabilizar as frequências relativas de
# tweets irrelevantes da planilha 'Treinamento'
li_irr = list()

for i in range(irrelevantes.shape[0]):
    li_irr += remove_stopwords(separa_emoji(cleanup(irrelevantes.Treinamento[irrelevantes.index[i]])))

serie_irr = pd.Series(li_irr)
irr_relat = serie_irr.value_counts(True)

In [640]:
probR = len(li_rel)/len(lista)

probI = len(li_irr)/len(lista)

In [641]:
test['Naive'] = ''

In [642]:
for n in range(test.shape[0]):
    probTdadoI = 1
    probTdadoR = 1

    tweet = remove_stopwords(separa_emoji(cleanup(test.Teste[n])))

    for p in tweet:
        if p in rel_relat:
            probTdadoR *= rel_relat[p]
    
    for p in tweet:
        if p in irr_relat:
            probTdadoI *= irr_relat[p]
    
    probRdadoT = probTdadoR * probR
    probIdadoT = probTdadoI * probI

    if probRdadoT > probIdadoT:
        test.Naive[n] = 1
    else:
        test.Naive[n] = 0

<ipython-input-642-d4dbc3ffc4d5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Naive[n] = 1
<ipython-input-642-d4dbc3ffc4d5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Naive[n] = 0


___

## Suavização de Laplace

In [643]:
palavras_possiveis = serie_twt.value_counts().shape[0]
qtd_rel = serie_rel.value_counts().shape[0]
qtd_irr = serie_irr.value_counts().shape[0]

In [644]:
rel_abs = serie_rel.value_counts()
irr_abs = serie_irr.value_counts()

In [645]:
test['Laplace'] = ''

In [652]:
for n in range(test.Teste.shape[0]):
    produtoR = 1
    tweet = remove_stopwords(separa_emoji(cleanup(test.Teste[n])))
    for p in tweet:
        if not p in rel_abs:
            rel_abs[p] = 0
        produtoR *= (rel_abs[p] + 1)/(qtd_rel + palavras_possiveis)

    produtoI = 1

    for p in tweet:
        if not p in irr_abs:
            irr_abs[p] = 0
        produtoI *= (irr_abs[p] + 1)/(qtd_irr + palavras_possiveis)

    probRdadoT = produtoR * probR
    probIdadoT = produtoI * probI

    if probRdadoT > probIdadoT:
        test.Laplace[n] = 1
    else:
        test.Laplace[n] = 0

<ipython-input-652-abc79c90dd25>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Laplace[n] = 1
<ipython-input-652-abc79c90dd25>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Laplace[n] = 0


___

## Verificação de performance

* **Sem a Suavização de Laplace**

In [654]:
b = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.Naive[i]:
        b += 1
print(f'- Exatidão: {b/test.shape[0]*100}%')

# PARA VERDADEIROS POSITIVOS
vpC = test.Classificação == 1
vpN = test.Naive == 1

# PARA FALSOS POSITIVOS
fpC = test.Classificação == 0
fpN = test.Naive == 1

# PARA VERDADEIROS NEGATIVOS
vnC = test.Classificação == 0
vnN = test.Naive == 0

# PARA FALSOS NEGATIVOS
fnC = test.Classificação == 1
fnN = test.Naive == 0

vp = (test.Teste.loc[vpC & vpN].count()/vpC.sum())*100
fp = (test.Teste.loc[fpC & fpN].count()/fpC.sum())*100
vn = (test.Teste.loc[vnC & vnN].count()/vnC.sum())*100
fn = (test.Teste.loc[fnC & fpN].count()/fnC.sum())*100

print(f'- Verdadeiros positivos (tweets relevantes classificados como relevantes): {vp:.1f}%\n'
      f'- Falsos positivos (tweets irrelevantes classificados como relevantes): {fp:.1f}%\n'
      f'- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): {vn:.1f}%\n'
      f'- Falsos negativos (tweets relevantes classificados como irrelevantes): {fn:.1f}%\n')



- Exatidão: 46.5%
- Verdadeiros positivos (tweets relevantes classificados como relevantes): 41.0%
- Falsos positivos (tweets irrelevantes classificados como relevantes): 48.0%
- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): 52.0%
- Falsos negativos (tweets relevantes classificados como irrelevantes): 41.0%



* **Com a Suavização de Laplace**

In [655]:
a = 0
for i in range(test.shape[0]):
    if test.Classificação[i] == test.Laplace[i]:
        a += 1
print(f'- Exatidão: {a/test.shape[0]*100}%')

# PARA VERDADEIROS POSITIVOS
vpC = test.Classificação == 1
vpL = test.Laplace == 1

# PARA FALSOS POSITIVOS
fpC = test.Classificação == 0
fpL = test.Laplace == 1

# PARA VERDADEIROS NEGATIVOS
vnC = test.Classificação == 0
vnL = test.Laplace == 0

# PARA FALSOS NEGATIVOS
fnC = test.Classificação == 1
fnL = test.Laplace == 0

vp = (test.Teste.loc[vpC & vpL].count()/vpC.sum())*100
fp = (test.Teste.loc[fpC & fpL].count()/fpC.sum())*100
vn = (test.Teste.loc[vnC & vnL].count()/vnC.sum())*100
fn = (test.Teste.loc[fnC & fpL].count()/fnC.sum())*100

print(f'- Verdadeiros positivos (tweets relevantes classificados como relevantes): {vp:.1f}%\n'
      f'- Falsos positivos (tweets irrelevantes classificados como relevantes): {fp:.1f}%\n'
      f'- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): {vn:.1f}%\n'
      f'- Falsos negativos (tweets relevantes classificados como irrelevantes): {fn:.1f}%\n')


- Exatidão: 69.5%
- Verdadeiros positivos (tweets relevantes classificados como relevantes): 96.0%
- Falsos positivos (tweets irrelevantes classificados como relevantes): 57.0%
- Verdadeiros negativos (tweets irrelevantes classificados como irrelevantes): 43.0%
- Falsos negativos (tweets relevantes classificados como irrelevantes): 96.0%



___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**